# 1. Chuẩn bị dữ liệu

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tkinter as tk

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.svm import OneClassSVM
from sklearn.impute import KNNImputer

Mô tả các features:

ID: Mã số định danh của khách hàng.\
Warehouse block: Kho chứa hàng của công ty vận chuyển, được chia theo kho A, B, C, D, F.\
Mode of shipment: Cách thức chuyển phát Ship, Flight and Road.\
Customer care calls: Số lượng cuộc gọi hối giao hàng.\
Customer rating: Thang đánh giá bởi khách hàng, 1 (rất không hài lòng) - 5 (rất hài lòng).\
Cost of the product: Giá (đơn vị tính: USD).\
Prior purchases: Số lần khách hàng đã nhờ vận chuyển trước đây.\
Product importance: Mức độ quan trọng của hàng hóa low, medium, high.\
Gender: Giới tính (M/F).\
Discount offered: Số tiền giảm giá cho hàng hóa.\
Weight in gms: Số kí của hàng hóa (đơn vị tính: gram).\
Reached on time (target): 0-giao đúng hẹn, 1-giao trễ hẹn.

In [ ]:
df = pd.read_csv("C:/Users/VivoBook/Downloads/ecom_ship.csv")
pd.set_option('display.max_columns', None)
df.head()


In [ ]:
# kích thước dataset
print('Before checking duplicated data:', df.shape)

df = df.drop_duplicates(keep='last')
print('After checking duplicated data:', df.shape)


In [ ]:
# kiểm tra missing value
df.info()


In [ ]:
# xóa cột không cần thiết
del df['ID']

# ép kiểu biến target về object
df[['Reached.on.Time_Y.N', 'Customer_rating']] = df[['Reached.on.Time_Y.N', 'Customer_rating']].astype('object')

# chuyển đổi đơn vị tính (g -> kg)
df['Weight_in_kg'] = df['Weight_in_gms'].apply(lambda x: x / 1000)
del df['Weight_in_gms']
df.insert(9, 'Weight_in_kg', df.pop('Weight_in_kg'))

df.head(5)

# 2. Tiền xử lí biến định lượng

In [ ]:
# thống kế mô tả cho biến định lượng
df.describe(include=['int64', 'float64'])

In [ ]:
df_quatitative = df[df.describe(include=['int64', 'float64']).columns]
df_quatitative.head(3)

In [ ]:
# kiểm tra outlier cho biến định lượng
def draw_box_plot(dataset, col_name):
    fig = plt.figure(figsize =(5, 3))
    plt.boxplot(dataset[col_name], vert=False)
    plt.title(f'Box plot for {col_name}', color='b')
    plt.show()


def find_outlier(dataset, col_name):
    minimum = dataset[col_name].min()
    maximum = dataset[col_name].max()
    median = dataset[col_name].median()
    
    q3, q1 = np.percentile(dataset[col_name], [75, 25])
    iqr = q3 - q1
    upperlimit = q3 + iqr * 1.5
    lowerlimit = q1 - iqr * 1.5   
    print(pd.DataFrame([[minimum, maximum], [lowerlimit, upperlimit]], index=['Min/Max value', 'Lower/Upper limit']))

    if minimum >= lowerlimit and maximum <= upperlimit:
        return f"Attribute {col_name} doesn't need to eliminate outliers."
    else:
        #count_row = ((dataset[col_name] < lowerlimit) | (dataset[col_name] > upperlimit)).sum()
        #dataset[col_name] = dataset[col_name].apply(lambda x: x if (x <= upperlimit and x >= lowerlimit) else median)

        count_row = 0
        for idx, row in dataset.iterrows():
            if row[col_name] > upperlimit or row[col_name] < lowerlimit:
                dataset.at[idx, col_name] = median
                count_row += 1

        draw_box_plot(dataset, col_name)
        return f'Attribute {col_name} was eliminated outliers!\n->{count_row} data points were changed by median.'


for col in df_quatitative.columns:
    print(draw_box_plot(df_quatitative, col))
    print(find_outlier(df_quatitative, col)) 
    print('\n______________________________________________________________________\n')


# 3. Tiền xử lí biến định tính

In [ ]:
# thống kế mô tả cho biến định tính
df.describe(include='object')


In [ ]:
df_qualitative = df[df.describe(include=['object']).columns]
df_qualitative.head(3)


In [ ]:
def draw_histogram(dataset, col_name, ax):
    ax.hist(dataset[col_name])
    ax.set_title(f'Histogram for {col_name}', color='r')
    ax.set_xlabel('Value')
    ax.set_ylabel('Frequency')

fig, axes = plt.subplots(3, 2, figsize=(12, 10)) # Tạo ma trận 3 hàng và 2 cột
axes = axes.flatten()                            # Làm phẳng mảng axes nếu nó có nhiều hàng và cột

# Duyệt qua từng cột và vẽ biểu đồ
for i, col in enumerate(df_qualitative.columns):
    draw_histogram(df_qualitative, col, axes[i])

plt.tight_layout()  # Điều chỉnh các biểu đồ sao cho không bị chồng lên nhau
plt.show()


In [ ]:
# Bước 1: Chuyển đổi các biến định tính thành dạng số (One-Hot Encoding)
encoder = OneHotEncoder(drop='first')  # 'drop' để tránh đa cộng tuyến (collinearity)
df_encoded = encoder.fit_transform(df_qualitative).toarray()
df_encoded

# Bước 2: Sử dụng OCSVM để phát hiện outliers
ocsvm = OneClassSVM(nu=0.1, kernel='rbf', gamma='scale')  # nu: tỷ lệ outliers dự đoán
ocsvm.fit(df_encoded)

# Dự đoán: -1 là outlier, 1 là normal
predictions = ocsvm.predict(df_encoded)
outlier_indexes = df.index[predictions == -1].tolist()
print("Predictions (1 = Normal, -1 = Outlier):", predictions)

# Chỉ thể hiện các outliers
outliers = df_qualitative[predictions == -1]
outliers.shape


In [ ]:
df = pd.concat([df_quatitative, df_qualitative], axis=1)
df.drop(index=outlier_indexes, inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

# 4. Dự đoán phân lớp

In [ ]:
df.info()

In [15]:
# 1. Tìm các cột có kiểu dữ liệu 'object' (định dạng chuỗi/định tính)
object_columns = df.select_dtypes(include=['object']).columns

# 2. Áp dụng One-Hot Encoding cho các cột định tính
encoder = OneHotEncoder(sparse_output=False)  # Đảm bảo không trả về dạng ma trận sparse
encoded_data = encoder.fit_transform(df[object_columns])

# 3. Chuyển kết quả One-Hot Encoding thành DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(object_columns))

# 4. Kết hợp DataFrame đã mã hóa vào DataFrame ban đầu theo chiều ngang
df_encoded = df.drop(columns=object_columns)              # Xóa các cột object ban đầu
df_encoded = pd.concat([df_encoded, encoded_df], axis=1)  # Kết hợp các cột đã mã hóa vào DataFrame ban đầu theo chiều ngang


In [16]:
# 5. Chia dữ liệu thành tập huấn luyện và kiểm tra
X = df_encoded.iloc[:, :-1]
y = df_encoded.iloc[:, -1]  

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# 6. Khởi tạo mô hình SVM (Support Vector Machine)
svm_model = SVC(kernel='poly')  # kernel = {rbf, poly, sigmoid, linear}

svm_model.fit(X_train, y_train)
y_pred = svm_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

print("\nClassification Report:")
print(classification_report(y_test, y_pred))